<a href="https://colab.research.google.com/github/HwangHanJae/ml-definitive-guide-pratice/blob/main/sklearn_model_selection_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

사이킷런의 model_selection 모듈
- 학습데이터, 테스트데이터 분리
- 교차 검증 분할 및 평가
- Estimator의 하이퍼 파라미터 튜닝을 위한 함수, 클래스 제공


In [2]:
#데이터를 분리하지 않으면 생기는 문제상황
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

#데이터 로드
iris = load_iris()
#모델 객체 생성
dt_clf = DecisionTreeClassifier()

train_data = iris.data
train_label = iris.target

#학습
dt_clf.fit(train_data, train_label)

#예측 

pred = dt_clf.predict(train_data)

#평가
print("예측 정확도 : ", accuracy_score(train_label, pred))

예측 정확도 :  1.0


이미 학습한 데이터를 기반으로 예측을 했기 때문에 정확도가 1.0 (100%)로 나옴
(이미 시험문제와 답을 알고 있는 시험을 봤다고 생각하면 됨)


train_test_split() - 학습데이터, 테스트데이터 분리
- test_size : 전체 데이터에서 테스트 데이터 세트의 크기를 얼마로 샘플링 할것인가
  - 기본값으로 0.25(25%)가 설정
- train_size : 전체 데이터에서 학습용 데이터 세트 크기를 얼마로 샘플링 할것인가
  - train_size 파라미터는 잘 사용하지 않음 
  - 대신 test_size를 사용
- shuffle : 데이터를 분리하기 전에 데이터를 미리 섞을지 결정
  - 기본값은 True
  - 데이터를 분산 시키기에 좀더 효율적
- random_state : 난수값

In [6]:
seed = 121
#테스트 데이터를 전체의 30% , 학습 데이터를 70%로 분리

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

dt_clf = DecisionTreeClassifier()
iris_data = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size=0.3, random_state=seed)

print("X_train 크기 : ", X_train.shape)
print("y_train 크기 : ", y_train.shape)
print("X_test 크기 : ", X_test.shape)
print("y_test 크기 : ", y_test.shape)

print("학습 데이터의 비율 : ", X_train.shape[0] / iris_data.data.shape[0])
print("테스트 데이터의 비율 : ", X_test.shape[0] / iris_data.data.shape[0])

#분리된 데이터로 학습
dt_clf.fit(X_train, y_train)

#예측
pred=dt_clf.predict(X_test)

#평가

print("예측 정확도 : ", accuracy_score(y_test, pred))

X_train 크기 :  (105, 4)
y_train 크기 :  (105,)
X_test 크기 :  (45, 4)
y_test 크기 :  (45,)
학습 데이터의 비율 :  0.7
테스트 데이터의 비율 :  0.3
예측 정확도 :  0.9555555555555556


전체 데이터의 크기는 150개로 각각 분리한 학습데이터는 105개, 테스트데이터는 45개 이다.

테스트데이터가 45개로 크지 않아 알고리즘의 정확한 예측 성능을 판단하기에는 어렵다.

과적합(Overfitting)의 가능성이 높다.
- 과적합 : 모델이 학습데이터에만 과도하게 최적화되어 실제 예측을 위한 데이터에서는 성능이 과도하게 떨어지는 것



교차검증

: 고정된 학습/테스트 데이터로만 평가를 하다보면 고정된 테스트데이터에만 편향되어 모델이 만들어지는 경우를 개선하기 위하여 사용

- 학습데이터, 검증데이터, 테스트데이터로 분리하여 평가
1. 분리된 학습/검증데이터로 교차검증을 진행
2. 남아있는 테스트데이터로 최종평가를 진행

KFold 교차검증

: k개의 폴드된 데이터 세트를 학습과 검증을 위한 데이터 세트로 변경하면서 k번 평가르 수행한뒤 이 k개의 평가를 평균한 결과를 가지고 예측 성능을 평가
- K는 일반적으로 5, 10을 사용, 5를 사용한다고 가정


In [8]:
#KFold 교차검증
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.datasets import load_iris
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier()

#5개의 폴드 세트로 분리하는 KFold 객체와 포드 세트별 정확도를 담을 리스트 객체 생성
#n_splits == k == 5
kfold = KFold(n_splits = 5)
cv_accuracy = []
print("붖꽃 데이터 세트 크기 : ", features.shape[0])

n_iter = 0
#KFold 객체의 split()메서드를 호출하면 폴드 별 학습용, 검증용 데이터의 로우 인덱스를 array로 반환
for train_index, val_index in kfold.split(features) :
  #kfold.split()으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
  X_train, X_val = features[train_index], features[val_index]
  y_train, y_val = label[train_index], label[val_index]

  #학습 및 예측
  dt_clf.fit(X_train, y_train)
  pred = dt_clf.predict(X_val)
  n_iter += 1

  #반복시 마다 정확도 측정
  accuracy = np.round(accuracy_score(y_val, pred), 4)
  train_size = X_train.shape[0]
  val_size = X_val.shape[0]

  print(f"\n#{n_iter} 교차검증 정확도 : {accuracy}, 학습 데이터 크기 : {train_size}, 검증 데이터 크기 : {val_size}")
  print(f"#{n_iter} 검증 데이터 인덱스 : {val_index}")

  cv_accuracy.append(accuracy)

#개별 iteration별 정확도를 합하여 평균 정확도 계산
print("\n## 평균 검증 정확도 : ", np.mean(cv_accuracy))

붖꽃 데이터 세트 크기 :  150

#1 교차검증 정확도 : 1.0, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#1 검증 데이터 인덱스 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차검증 정확도 : 1.0, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#2 검증 데이터 인덱스 : [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차검증 정확도 : 0.8667, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#3 검증 데이터 인덱스 : [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차검증 정확도 : 0.9333, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#4 검증 데이터 인덱스 : [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차검증 정확도 : 0.7333, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#5 검증 데이터 인덱스 : [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도 :  0.9066599999999999


Stratified KFold

: 불균형한 분포를 가진 label(target) 데이터 집합을 위한 KFold 방식
- 불균형한 분포를 가진 label 데이터 집합은 특정 label값이 매우 많거나, 적어서 값의 분포가 한쪽으로 치우치는것을 말합니다.

- 교차검증 진행할 때 특정 label집합이 한쪽으로 몰려있는 것을 방지함
- 학습/검증 데이터 세트에 각각 올바른 비율로 분배함
- split() 메서드를 사용시 학습데이터와 label데이터(학습)도 인자로 넣어줘야함

In [9]:
#불균형한 분포를 가진 label 상황(문제상황)

from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()

iris_df = pd.DataFrame(data=iris.data, columns = iris.feature_names)
iris_df['label'] = iris.target

print(iris_df['label'].value_counts())

2    50
1    50
0    50
Name: label, dtype: int64


label(0,1,2)값은 50개로 동일

In [10]:
#불균형한 분포를 가진 label 상황(문제상황)

#각 교차검증(KFold)시 생성되는 학습/검증 데이터 값의 분포도 확인

from sklearn.model_selection import KFold

kfold = KFold(n_splits=3)
n_iter = 0
for train_index, val_index in kfold.split(iris_df):
  n_iter += 1
  label_train = iris_df['label'].iloc[train_index]
  label_val = iris_df['label'].iloc[val_index]
  print(f"## 교차 검증 : {n_iter}")
  print(f"학습 레이블 데이터 분포 :\n", label_train.value_counts())
  print(f"검증 레이블 데이터 분포 :\n", label_val.value_counts())

## 교차 검증 : 1
학습 레이블 데이터 분포 :
 2    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 0    50
Name: label, dtype: int64
## 교차 검증 : 2
학습 레이블 데이터 분포 :
 2    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 1    50
Name: label, dtype: int64
## 교차 검증 : 3
학습 레이블 데이터 분포 :
 1    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 2    50
Name: label, dtype: int64


1,2 레이블이 학습할 때 0 레이블의 경우 전혀 학습을 하지 못함

또한 검증 레이블이 0밖에 없으므로 학습 모델은 절대 0을 예측하지 못함

In [11]:
#StratifiedKFold

from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)

n_iter = 0
for train_index, val_index in skf.split(iris_df, iris_df['label']):
  n_iter += 1
  label_train = iris_df['label'].iloc[train_index]
  label_val = iris_df['label'].iloc[val_index]
  print(f"## 교차 검증 {n_iter}")
  print(f"학습 레이블 데이터 분포 :\n", label_train.value_counts())
  print(f"검증 레이블 데이터 분포 :\n", label_val.value_counts())

## 교차 검증 1
학습 레이블 데이터 분포 :
 2    34
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 1    17
0    17
2    16
Name: label, dtype: int64
## 교차 검증 2
학습 레이블 데이터 분포 :
 1    34
2    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 2    17
0    17
1    16
Name: label, dtype: int64
## 교차 검증 3
학습 레이블 데이터 분포 :
 0    34
2    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 2    17
1    17
0    16
Name: label, dtype: int64


학습/검증 label 값의 분포도가 동일하게 할당되었음



In [14]:
#StratifiedKFold 교차검증
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.datasets import load_iris
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier()

#5개의 폴드 세트로 분리하는 KFold 객체와 포드 세트별 정확도를 담을 리스트 객체 생성
#n_splits == k == 5
skfold = StratifiedKFold(n_splits = 3)
cv_accuracy = []
print("붖꽃 데이터 세트 크기 : ", features.shape[0])

n_iter = 0
#KFold 객체의 split()메서드를 호출하면 폴드 별 학습용, 검증용 데이터의 로우 인덱스를 array로 반환
for train_index, val_index in skfold.split(features, label) :
  #kfold.split()으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
  X_train, X_val = features[train_index], features[val_index]
  y_train, y_val = label[train_index], label[val_index]

  #학습 및 예측
  dt_clf.fit(X_train, y_train)
  pred = dt_clf.predict(X_val)
  n_iter += 1

  #반복시 마다 정확도 측정
  accuracy = np.round(accuracy_score(y_val, pred), 4)
  train_size = X_train.shape[0]
  val_size = X_val.shape[0]

  print(f"\n#{n_iter} 교차검증 정확도 : {accuracy}, 학습 데이터 크기 : {train_size}, 검증 데이터 크기 : {val_size}")
  print(f"#{n_iter} 검증 데이터 인덱스 : {val_index}")

  cv_accuracy.append(accuracy)

#개별 iteration별 정확도를 합하여 평균 정확도 계산
print("\n## 평균 검증 정확도 : ", np.mean(cv_accuracy))

붖꽃 데이터 세트 크기 :  150

#1 교차검증 정확도 : 0.98, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
#1 검증 데이터 인덱스 : [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

#2 교차검증 정확도 : 0.94, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
#2 검증 데이터 인덱스 : [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

#3 교차검증 정확도 : 1.0, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
#3 검증 데이터 인덱스 : [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도 :  0.9733333333333333


cross_val_score()
: 교차 검증을 좀더 편리하게 수행할 수 있는 API
- 위의 일련의 과정들을 한번에 수행해줌
  위의 과정들 :
    1. 폴드 세트를 설정함
    2. for 루프에서 반복으로 학습 및 테스트 데이터의 인덱스를 추출함
    3. 반복적으로 학습과 예측을 수행하고 예측 성능을 반환
- 파라미터
  - estimator : 분류 알고리즘 클래스(Classifier), 회귀 알고리즘 클래스(Regressor)
  - X : 피처 데이터 세트
  - y : 레이블 데이터 세트
  - scoring : 예측 성능 평가 지표
  - cv : 교차 검증 폴드 수
  - return 값 : scoring 파라미터로 지정된 성능 지표 측정값을 배열 형태로 반환
- 분류 알고리즘 입력시 StratifiedKFold 분할
- 회귀 알고리즘 입력시 KFold로 분할
  - 회귀 알고리즘 경우는 특정 수치(값)으로 예측하기 때문에 StraitifedKFold가 필요하지 않음


In [20]:
#cross_val_score()

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data= iris_data.data
label = iris_data.target

#성능지표(scoring)은 accurcay(정확도), 교차 검증 세트는 3개

scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3)
print("교차 검증별 정확도 : ", np.round(scores, 4))
print("평균 검증 정확도 : ", np.round(np.mean(scores),4))

교차 검증별 정확도 :  [0.98 0.94 0.98]
평균 검증 정확도 :  0.9667


cross_validate()는 여러개의 평가지표를 반환 할수 있음 또한 학습 데이터에 대한 성능 평가 지표와 수행 시간도 함께 제공